# Taller Práctico: Fundamentos de LangChain para Procesamiento de Documentos

## Objetivos Educativos

- Comprender los conceptos fundamentales de LangChain y su ecosistema
- Aprender a integrar modelos de lenguaje en aplicaciones prácticas
- Dominar el uso de cargadores de documentos y transformadores de datos
- Implementar sistemas de recuperación de información basados en embeddings
- Construir cadenas de procesamiento secuenciales y complejas
- Desarrollar habilidades para optimizar consultas mediante compresión contextual

## Resultados Esperados

- Capacidad para cargar y procesar diferentes tipos de documentos (PDF, texto, fuentes externas)
- Habilidad para crear y gestionar plantillas de prompts reutilizables
- Competencia en la construcción de bases de datos vectoriales para búsqueda semántica
- Experiencia práctica en el desarrollo de cadenas de procesamiento con múltiples pasos
- Conocimiento aplicado sobre serialización y persistencia de componentes
- Comprensión de técnicas de optimización para recuperación de información

---

## Estructura del Taller

Este taller está dividido en dos módulos principales:

**Módulo 1: Conectores de Datos en LangChain**
- Cargadores de documentos
- Transformación y segmentación
- Embeddings y almacenamiento vectorial
- Compresión contextual

**Módulo 2: Cadenas de Procesamiento**
- Cadenas simples y secuenciales
- Integración de múltiples componentes
- Casos de uso prácticos

---

## Requisitos Técnicos

- Google Colab (entorno de ejecución)
- Acceso a Google Generative AI (API gratuita)
- Conocimientos básicos de Python

## Configuración Inicial del Entorno

Antes de comenzar, necesitamos instalar las librerías necesarias y configurar nuestro acceso a la API de Google Generative AI.

### ¿Por qué LangChain?

LangChain es un framework que simplifica la construcción de aplicaciones basadas en modelos de lenguaje. Permite conectar LLMs con fuentes de datos externas, crear cadenas de procesamiento complejas y construir sistemas que pueden razonar sobre información específica.

En este taller utilizaremos la API gratuita de Google Generative AI (Gemini), disponible directamente desde Google Colab.

In [1]:
# Instalación de paquetes necesarios
!pip install -q langchain langchain-google-genai langchain-community
!pip install -q chromadb pypdf faiss-cpu tiktoken
!pip install -q wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypro

In [21]:
# Configuración de la API de Google Generative AI
# Para obtener tu API key gratuita, visita: https://makersuite.google.com/app/apikey

import os
from google.colab import userdata

# Guarda tu API key en los secretos de Colab con el nombre 'GOOGLE_API_KEY'
# O descomenta la siguiente línea y reemplaza con tu clave
# os.environ['GOOGLE_API_KEY'] = 'tu-api-key-aqui'

# Si guardaste la clave en secretos de Colab:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY3')

print("Configuración completada exitosamente")

Configuración completada exitosamente


---

# Módulo 1: Conectores de Datos en LangChain

## 1.1 Cargadores de Documentos

Los cargadores de documentos (Document Loaders) son componentes fundamentales en LangChain que permiten importar datos desde múltiples fuentes. Cada cargador está diseñado para trabajar con un formato específico y retorna objetos Document que contienen el texto y metadatos asociados.

### Concepto Clave: Objeto Document

Un Document en LangChain tiene dos componentes principales:
- **page_content**: El contenido textual del documento
- **metadata**: Información adicional (fuente, fecha, página, etc.)

### Ejemplo Práctico: Carga de Documentos de Texto

Comenzaremos con el cargador más simple: cargar archivos de texto plano.

In [3]:
# Creamos un archivo de texto de ejemplo sobre legislación ambiental
contenido_muestra = """
Ley de Protección de Recursos Hídricos - Artículo 23

Toda empresa industrial que opere en zonas cercanas a cuerpos de agua dulce
debe implementar sistemas de tratamiento de aguas residuales que cumplan con
los estándares establecidos en el Anexo B de esta normativa.

Las sanciones por incumplimiento pueden alcanzar hasta el 15% de los ingresos
anuales de la empresa infractora. Se establece un período de adaptación de
24 meses desde la publicación de esta ley.

Los municipios deberán presentar informes trimestrales sobre la calidad del
agua en sus jurisdicciones al Ministerio de Medio Ambiente.
"""

# Guardar el contenido en un archivo
with open('legislacion_ambiental.txt', 'w', encoding='utf-8') as f:
    f.write(contenido_muestra)

print("Archivo de ejemplo creado exitosamente")

Archivo de ejemplo creado exitosamente


In [4]:
# Importar el cargador de documentos de texto
from langchain_community.document_loaders import TextLoader

# Cargar el documento
loader = TextLoader('legislacion_ambiental.txt', encoding='utf-8')
documentos = loader.load()

# Explorar el contenido cargado
print(f"Número de documentos cargados: {len(documentos)}")
print(f"\nContenido del documento:")
print(documentos[0].page_content)
print(f"\nMetadatos:")
print(documentos[0].metadata)

Número de documentos cargados: 1

Contenido del documento:

Ley de Protección de Recursos Hídricos - Artículo 23

Toda empresa industrial que opere en zonas cercanas a cuerpos de agua dulce 
debe implementar sistemas de tratamiento de aguas residuales que cumplan con 
los estándares establecidos en el Anexo B de esta normativa.

Las sanciones por incumplimiento pueden alcanzar hasta el 15% de los ingresos 
anuales de la empresa infractora. Se establece un período de adaptación de 
24 meses desde la publicación de esta ley.

Los municipios deberán presentar informes trimestrales sobre la calidad del 
agua en sus jurisdicciones al Ministerio de Medio Ambiente.


Metadatos:
{'source': 'legislacion_ambiental.txt'}


### Reflexión 1

Pregunta: ¿Qué diferencia existe entre el atributo `page_content` y `metadata` en un objeto Document? ¿Por qué es útil separar esta información?

---

## 1.2 Caso Práctico: Carga de Documentos PDF y Resumen con LLM

Los archivos PDF son uno de los formatos más comunes para documentos técnicos y legales. LangChain proporciona cargadores especializados para extraer texto de estos archivos.

### Flujo de Trabajo
1. Cargar el PDF utilizando PyPDFLoader
2. Extraer el contenido
3. Enviar el contenido a un LLM para generar un resumen

In [5]:
# Crear un PDF de ejemplo usando reportlab
!pip install -q reportlab

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def crear_pdf_ejemplo():
    c = canvas.Canvas("informe_seguridad_laboral.pdf", pagesize=letter)
    width, height = letter

    # Contenido del informe
    c.setFont("Helvetica-Bold", 16)
    c.drawString(100, height - 100, "Informe de Seguridad Laboral 2024")

    c.setFont("Helvetica", 12)
    y_position = height - 150

    contenido = [
        "Resumen Ejecutivo",
        "",
        "Durante el último trimestre se registraron 47 incidentes menores",
        "y 3 incidentes mayores en las instalaciones de producción.",
        "",
        "Principales hallazgos:",
        "- 68% de los incidentes ocurrieron en el turno nocturno",
        "- El área de maquinaria pesada presentó el mayor número de reportes",
        "- Se identificó falta de capacitación en protocolos de emergencia",
        "",
        "Recomendaciones:",
        "1. Implementar programa de capacitación trimestral obligatoria",
        "2. Incrementar supervisión en turno nocturno",
        "3. Renovar señalización de seguridad en áreas críticas",
        "4. Realizar auditorías mensuales de equipos de protección",
    ]

    for linea in contenido:
        c.drawString(100, y_position, linea)
        y_position -= 20

    c.save()
    print("PDF de ejemplo creado: informe_seguridad_laboral.pdf")

crear_pdf_ejemplo()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.5 MB/s eta 0:00:00
PDF de ejemplo creado: informe_seguridad_laboral.pdf


In [6]:
# Cargar el PDF utilizando PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader

# Inicializar el cargador
loader = PyPDFLoader("informe_seguridad_laboral.pdf")
paginas = loader.load()

# Explorar el contenido
print(f"Número de páginas: {len(paginas)}\n")
print("Contenido de la primera página:")
print(paginas[0].page_content)
print(f"\nMetadatos de la página:")
print(paginas[0].metadata)

Número de páginas: 1

Contenido de la primera página:
Informe de Seguridad Laboral 2024
Resumen Ejecutivo
Durante el último trimestre se registraron 47 incidentes menores
y 3 incidentes mayores en las instalaciones de producción.
Principales hallazgos:
- 68% de los incidentes ocurrieron en el turno nocturno
- El área de maquinaria pesada presentó el mayor número de reportes
- Se identificó falta de capacitación en protocolos de emergencia
Recomendaciones:
1. Implementar programa de capacitación trimestral obligatoria
2. Incrementar supervisión en turno nocturno
3. Renovar señalización de seguridad en áreas críticas
4. Realizar auditorías mensuales de equipos de protección

Metadatos de la página:
{'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': 'ReportLab PDF Library - www.reportlab.com', 'creationdate': '2025-11-07T07:24:32+00:00', 'author': 'anonymous', 'keywords': '', 'moddate': '2025-11-07T07:24:32+00:00', 'subject': 'unspecified', 'title': 'untitled', 'trapped'

In [22]:
# Inicializar el modelo de lenguaje de Google
from langchain_google_genai import ChatGoogleGenerativeAI

# Configurar el modelo Gemini (gratuito desde Colab)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,  # Menor temperatura = respuestas más consistentes
    convert_system_message_to_human=True
)

In [19]:
# Crear un prompt para resumir el documento
from langchain.prompts import PromptTemplate

template_resumen = """
Eres un analista de seguridad laboral. Lee el siguiente informe y genera un resumen
ejecutivo de máximo 3 párrafos que destaque:
1. Los datos más relevantes
2. Los principales problemas identificados
3. Las acciones recomendadas

Informe:
{documento}

Resumen ejecutivo:
"""

prompt = PromptTemplate(
    input_variables=["documento"],
    template=template_resumen
)

# Combinar el contenido de todas las páginas
contenido_completo = "\n".join([pagina.page_content for pagina in paginas])

# Formatear el prompt con el documento
prompt_formateado = prompt.format(documento=contenido_completo)

# Generar el resumen
resumen = llm.invoke(prompt_formateado)
print("RESUMEN GENERADO:")
print("="*60)
print(resumen.content)

RESUMEN GENERADO:
**Resumen Ejecutivo de Seguridad Laboral 2024**

El último trimestre, las instalaciones de producción registraron un total de 50 incidentes, de los cuales 47 fueron menores y 3 de carácter mayor. Un dato crítico es que el 68% de estos incidentes ocurrieron durante el turno nocturno, siendo el área de maquinaria pesada la que concentró el mayor número de reportes, lo que indica puntos de riesgo específicos y horarios de mayor vulnerabilidad.

El análisis de estos hallazgos revela una preocupación principal: la **falta de capacitación adecuada en protocolos de emergencia**. Esta deficiencia, combinada con la alta incidencia en el turno nocturno y en áreas de alto riesgo como la maquinaria pesada, sugiere la necesidad urgente de reforzar las medidas preventivas y la supervisión para mitigar futuros incidentes, especialmente los de mayor gravedad.

Para abordar estas problemáticas de manera efectiva, se proponen las siguientes acciones inmediatas: implementar un programa 

### Análisis del Código Anterior

En el bloque anterior hemos realizado varios pasos importantes:

1. **Inicialización del LLM**: Configuramos el modelo Gemini con temperatura 0.3 para obtener respuestas más determinísticas
2. **PromptTemplate**: Creamos una plantilla reutilizable que define la estructura de nuestra petición
3. **Formateo**: Insertamos el contenido del documento en la plantilla
4. **Invocación**: Enviamos el prompt al modelo y obtenemos la respuesta

### Reflexión 2

Pregunta: ¿Por qué es útil usar PromptTemplate en lugar de simplemente concatenar strings? Piensa en escenarios donde necesites procesar múltiples documentos.

---

## 1.3 Cargadores con Integraciones Externas

LangChain soporta una amplia variedad de fuentes de datos. Exploraremos tres cargadores comunes para fuentes externas.

### A) Cargador de Wikipedia

Wikipedia es una fuente valiosa de información estructurada. El cargador de Wikipedia permite extraer artículos completos con un solo comando.

In [9]:
# Cargar información desde Wikipedia
from langchain_community.document_loaders import WikipediaLoader

# Buscar información sobre energía solar
loader = WikipediaLoader(
    query="Energía solar fotovoltaica",
    lang="es",  # Idioma español
    load_max_docs=2  # Limitar a 2 documentos
)

documentos_wiki = loader.load()

# Explorar el contenido
print(f"Documentos cargados: {len(documentos_wiki)}\n")
print(f"Título: {documentos_wiki[0].metadata['title']}")
print(f"Fuente: {documentos_wiki[0].metadata['source']}")
print(f"\nPrimeros 500 caracteres:")
print(documentos_wiki[0].page_content[:500])

Documentos cargados: 2

Título: Energía solar fotovoltaica
Fuente: https://es.wikipedia.org/wiki/Energ%C3%ADa_solar_fotovoltaica

Primeros 500 caracteres:
La energía solar fotovoltaica es una fuente de energía renovable que permite la producción de electricidad a partir de la radiación solar.​ El proceso se realiza mediante dispositivos semiconductores llamados células fotovoltaicas, que convierten directamente la energía lumínica en corriente eléctrica por medio del efecto fotovoltaico.​  
Existen diferentes tecnologías de fabricación de células, entre las que destacan:  
Células de silicio cristalino, basadas en obleas de silicio monocristalino 


### B) Cargador de URLs Web

Para extraer contenido de páginas web, podemos usar el WebBaseLoader.

In [10]:
# Instalar dependencia para web scraping
!pip install -q beautifulsoup4

from langchain_community.document_loaders import WebBaseLoader

# Cargar contenido de una página web
# Ejemplo: página de documentación oficial de Python sobre tipos de datos
loader = WebBaseLoader("https://docs.python.org/3/library/stdtypes.html")
documentos_web = loader.load()

print(f"Documentos cargados: {len(documentos_web)}")
print(f"Fuente: {documentos_web[0].metadata['source']}")
print(f"\nPrimeros 400 caracteres del contenido:")
print(documentos_web[0].page_content[:400])

Documentos cargados: 1
Fuente: https://docs.python.org/3/library/stdtypes.html

Primeros 400 caracteres del contenido:

















Built-in Types — Python 3.14.0 documentation




















































    Theme
    
Auto
Light
Dark



Table of Contents

Built-in Types
Truth Value Testing
Boolean Operations — and, or, not
Comparisons
Numeric Types — int, float, complex
Bitwise Operations on Integer Types
Additional Methods on Integer Types
Additional Methods on Float
Additional Metho


### C) Cargador de Archivos CSV

Los archivos CSV son fundamentales para trabajar con datos tabulares. Veamos cómo cargarlos y procesarlos.

In [11]:
# Crear un CSV de ejemplo con datos de consumo energético
import csv

datos_energia = [
    ['Mes', 'Consumo_kWh', 'Costo_USD', 'Fuente'],
    ['Enero', '4520', '678', 'Red eléctrica'],
    ['Febrero', '4100', '615', 'Red eléctrica'],
    ['Marzo', '3890', '583', 'Red eléctrica'],
    ['Abril', '3200', '480', 'Mixta (70% red, 30% solar)'],
    ['Mayo', '2800', '420', 'Mixta (50% red, 50% solar)'],
    ['Junio', '2500', '375', 'Mixta (40% red, 60% solar)'],
]

with open('consumo_energetico.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(datos_energia)

print("Archivo CSV creado exitosamente")

Archivo CSV creado exitosamente


In [12]:
# Cargar el archivo CSV
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(
    file_path='consumo_energetico.csv',
    encoding='utf-8',
    csv_args={'delimiter': ','}  # Especificar el delimitador
)

documentos_csv = loader.load()

# Explorar los documentos cargados
print(f"Número de filas cargadas: {len(documentos_csv)}\n")
print("Primer registro:")
print(documentos_csv[0].page_content)
print("\nTercer registro:")
print(documentos_csv[2].page_content)

Número de filas cargadas: 6

Primer registro:
Mes: Enero
Consumo_kWh: 4520
Costo_USD: 678
Fuente: Red eléctrica

Tercer registro:
Mes: Marzo
Consumo_kWh: 3890
Costo_USD: 583
Fuente: Red eléctrica


### Reflexión 3

Pregunta: ¿Qué ventajas tiene utilizar cargadores especializados en lugar de leer archivos manualmente con Python estándar? Considera aspectos como metadatos, estandarización y escalabilidad.

---

## 1.4 Transformación de Documentos

Los documentos cargados suelen ser demasiado largos para procesarlos en una sola solicitud al LLM. La división (splitting) de documentos es crucial para:

1. **Respetar límites de contexto**: Los LLMs tienen límites en la cantidad de tokens que pueden procesar
2. **Mejorar la precisión**: Fragmentos más pequeños permiten búsquedas más específicas
3. **Optimizar costos**: Enviar solo la información relevante reduce el uso de tokens

### Tipos de Divisores

LangChain ofrece múltiples estrategias de división:
- **CharacterTextSplitter**: Divide por número de caracteres
- **RecursiveCharacterTextSplitter**: Intenta mantener párrafos juntos
- **TokenTextSplitter**: Divide por número de tokens

In [13]:
# Crear un documento extenso para demostrar la división
texto_largo = """
Manual de Procedimientos de Mantenimiento Industrial

Capítulo 1: Mantenimiento Preventivo de Maquinaria Pesada

El mantenimiento preventivo es fundamental para garantizar la operatividad continua de los equipos.
Se debe realizar una inspección visual diaria antes de iniciar operaciones. Verificar niveles de
fluidos hidráulicos, aceite de motor y refrigerante.

Capítulo 2: Protocolos de Seguridad

Todo el personal debe utilizar equipo de protección personal completo. Las áreas de trabajo deben
estar debidamente señalizadas. Se prohíbe el acceso a personal no autorizado durante operaciones
de mantenimiento. Los candados de seguridad deben aplicarse en todos los puntos de energía.

Capítulo 3: Calibración de Instrumentos

La calibración debe realizarse trimestralmente por personal certificado. Los instrumentos de medición
deben almacenarse en condiciones controladas de temperatura y humedad. Mantener registros detallados
de todas las calibraciones realizadas.

Capítulo 4: Gestión de Repuestos

El inventario de repuestos críticos debe revisarse mensualmente. Establecer puntos de reorden basados
en tiempos de entrega de proveedores. Mantener stock de seguridad para componentes de alta rotación.
"""

# Usar RecursiveCharacterTextSplitter para mantener coherencia semántica
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Tamaño máximo de cada fragmento en caracteres
    chunk_overlap=50,  # Superposición entre fragmentos para mantener contexto
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Prioridad de separadores
)

# Dividir el texto
fragmentos = text_splitter.create_documents([texto_largo])

# Explorar los fragmentos creados
print(f"Documento original: {len(texto_largo)} caracteres")
print(f"Fragmentos generados: {len(fragmentos)}\n")

for i, fragmento in enumerate(fragmentos):
    print(f"--- Fragmento {i+1} ({len(fragmento.page_content)} caracteres) ---")
    print(fragmento.page_content[:150] + "...")
    print()

Documento original: 1218 caracteres
Fragmentos generados: 5

--- Fragmento 1 (111 caracteres) ---
Manual de Procedimientos de Mantenimiento Industrial

Capítulo 1: Mantenimiento Preventivo de Maquinaria Pesada...

--- Fragmento 2 (288 caracteres) ---
El mantenimiento preventivo es fundamental para garantizar la operatividad continua de los equipos. 
Se debe realizar una inspección visual diaria ant...

--- Fragmento 3 (288 caracteres) ---
Todo el personal debe utilizar equipo de protección personal completo. Las áreas de trabajo deben 
estar debidamente señalizadas. Se prohíbe el acceso...

--- Fragmento 4 (284 caracteres) ---
Capítulo 3: Calibración de Instrumentos

La calibración debe realizarse trimestralmente por personal certificado. Los instrumentos de medición 
deben ...

--- Fragmento 5 (237 caracteres) ---
Capítulo 4: Gestión de Repuestos

El inventario de repuestos críticos debe revisarse mensualmente. Establecer puntos de reorden basados 
en tiempos de...



### Análisis de Parámetros del Text Splitter

- **chunk_size**: Define el tamaño máximo de cada fragmento. Un valor muy pequeño puede romper el contexto, muy grande puede exceder límites del modelo.
- **chunk_overlap**: Crea superposición entre fragmentos para evitar perder información en los límites. Típicamente entre 10-20% del chunk_size.
- **separators**: Lista ordenada por prioridad. El divisor intentará primero dividir por párrafos (\\n\\n), luego líneas (\\n), después oraciones (. ), etc.

---

## 1.5 Incrustación de Texto y Creación de Embeddings

Los embeddings son representaciones vectoriales de texto que capturan su significado semántico. Dos textos con significados similares tendrán embeddings cercanos en el espacio vectorial.

### ¿Por qué usar embeddings?

- Permiten búsqueda semántica (por significado, no solo palabras clave)
- Facilitan la comparación de similitud entre documentos
- Son la base de sistemas RAG (Retrieval Augmented Generation)

In [24]:
# Configurar el modelo de embeddings de Google
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Inicializar el modelo de embeddings (gratuito con API de Google)
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001"
)

# Crear embeddings para diferentes textos relacionados con manufactura
textos_ejemplo = [
    "El proceso de soldadura requiere temperaturas superiores a 1500 grados centígrados",
    "La soldadura de metales necesita calor extremadamente alto para fundir el material",
    "Los protocolos de seguridad establecen el uso obligatorio de casco y guantes",
    "El mantenimiento preventivo reduce costos operativos en un 30%"
]

# Generar embeddings
embeddings = embeddings_model.embed_documents(textos_ejemplo)

# Analizar las dimensiones
print(f"Número de textos: {len(textos_ejemplo)}")
print(f"Dimensiones del embedding: {len(embeddings[0])}")
print(f"\nPrimeros 10 valores del embedding del texto 1:")
print(embeddings[0][:10])

Número de textos: 4
Dimensiones del embedding: 3072

Primeros 10 valores del embedding del texto 1:
[-0.0049664913676679134, 0.008457482792437077, 0.026392078027129173, -0.05674704909324646, -0.001503984211012721, 0.045801643282175064, 0.018904773518443108, -0.006415930576622486, -0.022402046248316765, 0.008778405375778675]


In [25]:
# Calcular similitud coseno entre embeddings para verificar relaciones semánticas
import numpy as np

def similitud_coseno(vec1, vec2):
    """Calcula la similitud coseno entre dos vectores"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Comparar similitudes
print("Matriz de similitudes:\n")
print("Textos comparados:")
for i, texto in enumerate(textos_ejemplo):
    print(f"{i+1}. {texto[:60]}...")

print("\n")
for i in range(len(embeddings)):
    for j in range(i+1, len(embeddings)):
        sim = similitud_coseno(embeddings[i], embeddings[j])
        print(f"Similitud entre texto {i+1} y texto {j+1}: {sim:.4f}")

print("\n¿Qué observamos?")
print("- Los textos 1 y 2 tienen alta similitud (hablan de soldadura)")
print("- Los textos 3 y 4 son diferentes entre sí y diferentes a 1-2")

Matriz de similitudes:

Textos comparados:
1. El proceso de soldadura requiere temperaturas superiores a 1...
2. La soldadura de metales necesita calor extremadamente alto p...
3. Los protocolos de seguridad establecen el uso obligatorio de...
4. El mantenimiento preventivo reduce costos operativos en un 3...


Similitud entre texto 1 y texto 2: 0.9012
Similitud entre texto 1 y texto 3: 0.7285
Similitud entre texto 1 y texto 4: 0.7582
Similitud entre texto 2 y texto 3: 0.7302
Similitud entre texto 2 y texto 4: 0.7593
Similitud entre texto 3 y texto 4: 0.7503

¿Qué observamos?
- Los textos 1 y 2 tienen alta similitud (hablan de soldadura)
- Los textos 3 y 4 son diferentes entre sí y diferentes a 1-2


---

## 1.6 Almacenamiento de Vectores en una Base de Datos Vectorial

Las bases de datos vectoriales están optimizadas para almacenar y buscar embeddings de manera eficiente. Utilizaremos Chroma, una base de datos vectorial de código abierto.

### Flujo de Trabajo

1. Cargar documentos
2. Dividirlos en fragmentos
3. Generar embeddings de cada fragmento
4. Almacenar en la base de datos vectorial
5. Realizar búsquedas por similitud semántica

In [26]:
# Crear una colección de documentos sobre protocolos industriales
documentos_protocolos = [
    "Los equipos de protección personal deben inspeccionarse antes de cada turno. Cascos, guantes y gafas son obligatorios en la planta de producción.",
    "El mantenimiento preventivo de las máquinas CNC debe realizarse cada 500 horas de operación. Incluye lubricación de ejes y calibración de herramientas.",
    "Los procedimientos de soldadura MIG requieren gas de protección argón-CO2 en proporción 80-20. La temperatura del arco debe mantenerse entre 1400-1600°C.",
    "El control de calidad estadístico utiliza muestreo aleatorio cada 100 unidades. Los límites de tolerancia están definidos en la especificación técnica ET-045.",
    "La gestión de inventarios se basa en el sistema Kanban. Las tarjetas de reposición se activan cuando el stock alcanza el 30% del nivel máximo.",
    "Los residuos peligrosos deben almacenarse en contenedores etiquetados según norma NOM-052. La disposición final la realiza una empresa certificada.",
    "El proceso de templado del acero requiere calentamiento a 850°C seguido de enfriamiento rápido en aceite. El tiempo de permanencia es crítico para la dureza final.",
    "Las auditorías de seguridad se realizan trimestralmente por un equipo externo. Los hallazgos deben documentarse y corregirse en 30 días máximo."
]

# Dividir los documentos en fragmentos más pequeños
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20
)

docs_divididos = splitter.create_documents(documentos_protocolos)
print(f"Fragmentos totales generados: {len(docs_divididos)}")

Fragmentos totales generados: 12


In [27]:
# Crear una base de datos vectorial con Chroma
from langchain_community.vectorstores import Chroma

# Crear el vector store (esto genera automáticamente los embeddings)
vectorstore = Chroma.from_documents(
    documents=docs_divididos,
    embedding=embeddings_model,
    collection_name="protocolos_industriales"
)

print("Base de datos vectorial creada exitosamente")
print(f"Documentos almacenados: {vectorstore._collection.count()}")

Base de datos vectorial creada exitosamente
Documentos almacenados: 12


In [28]:
# Realizar búsquedas por similitud semántica
consulta = "¿Cuáles son los requisitos de temperatura para procesos térmicos?"

# Buscar los 3 documentos más relevantes
documentos_relevantes = vectorstore.similarity_search(consulta, k=3)

print(f"Consulta: {consulta}\n")
print("Documentos más relevantes encontrados:\n")

for i, doc in enumerate(documentos_relevantes):
    print(f"--- Resultado {i+1} ---")
    print(doc.page_content)
    print()

Consulta: ¿Cuáles son los requisitos de temperatura para procesos térmicos?

Documentos más relevantes encontrados:

--- Resultado 1 ---
El proceso de templado del acero requiere calentamiento a 850°C seguido de enfriamiento rápido en aceite. El tiempo de permanencia es crítico para la

--- Resultado 2 ---
mantenerse entre 1400-1600°C.

--- Resultado 3 ---
es crítico para la dureza final.



In [29]:
# Búsqueda con scores de similitud
resultados_con_scores = vectorstore.similarity_search_with_score(consulta, k=3)

print("Resultados con puntuación de similitud:\n")
for doc, score in resultados_con_scores:
    print(f"Score: {score:.4f}")
    print(f"Contenido: {doc.page_content}")
    print("-" * 60)

Resultados con puntuación de similitud:

Score: 0.3260
Contenido: El proceso de templado del acero requiere calentamiento a 850°C seguido de enfriamiento rápido en aceite. El tiempo de permanencia es crítico para la
------------------------------------------------------------
Score: 0.3541
Contenido: mantenerse entre 1400-1600°C.
------------------------------------------------------------
Score: 0.4309
Contenido: es crítico para la dureza final.
------------------------------------------------------------


### Reflexión 4

Pregunta: Observa los resultados de la búsqueda. ¿Por qué el sistema recuperó esos documentos específicos aunque la consulta no contenga las palabras exactas? ¿Qué ventaja tiene esto sobre una búsqueda por palabras clave tradicional?

---

## 1.7 Compresión y Optimización de Consultas mediante LLMs

Cuando trabajamos con documentos largos, no siempre es eficiente enviar todo el contexto recuperado al LLM. La compresión contextual permite extraer solo la información relevante de los documentos recuperados.

### Contextual Compression Retriever

Este componente:
1. Realiza una búsqueda inicial (recupera documentos potencialmente relevantes)
2. Comprime cada documento para extraer solo las partes relevantes a la consulta
3. Retorna fragmentos más pequeños y específicos

In [30]:
# Configurar un retriever con compresión contextual
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Crear el compresor que usa el LLM para extraer información relevante
compressor = LLMChainExtractor.from_llm(llm)

# Crear el retriever base (búsqueda vectorial estándar)
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# Combinar con compresión
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

print("Retriever con compresión configurado exitosamente")

Retriever con compresión configurado exitosamente


In [31]:
# Comparar búsqueda estándar vs búsqueda con compresión
consulta_especifica = "¿Cuál es la frecuencia del mantenimiento preventivo?"

print("="*70)
print("BÚSQUEDA ESTÁNDAR (sin compresión)")
print("="*70)
docs_estandar = base_retriever.get_relevant_documents(consulta_especifica)
for i, doc in enumerate(docs_estandar[:2]):
    print(f"\n--- Documento {i+1} ---")
    print(doc.page_content)

print("\n\n" + "="*70)
print("BÚSQUEDA CON COMPRESIÓN CONTEXTUAL")
print("="*70)
docs_comprimidos = compression_retriever.get_relevant_documents(consulta_especifica)
for i, doc in enumerate(docs_comprimidos[:2]):
    print(f"\n--- Documento comprimido {i+1} ---")
    print(doc.page_content)

BÚSQUEDA ESTÁNDAR (sin compresión)

--- Documento 1 ---
El mantenimiento preventivo de las máquinas CNC debe realizarse cada 500 horas de operación. Incluye lubricación de ejes y calibración de

--- Documento 2 ---
y calibración de herramientas.


BÚSQUEDA CON COMPRESIÓN CONTEXTUAL


/tmp/ipython-input-3819528289.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_estandar = base_retriever.get_relevant_documents(consulta_especifica)



--- Documento comprimido 1 ---
cada 500 horas de operación.


### Ventajas de la Compresión Contextual

1. **Reducción de tokens**: Envía solo información relevante al LLM final
2. **Mayor precisión**: Elimina ruido y contexto innecesario
3. **Optimización de costos**: Menos tokens procesados = menor costo en APIs de pago
4. **Mejor calidad de respuesta**: El modelo se enfoca en información pertinente

### Reflexión 5

Ejercicio: Prueba con otra consulta como "¿Qué normas regulan el manejo de residuos?" y observa cómo cambia el contenido recuperado con y sin compresión.

---

# Módulo 2: Cadenas de Procesamiento en LangChain

## 2.1 Definición de Cadenas y Primera Cadena Secuencial Simple

Las cadenas (chains) son componentes que permiten combinar múltiples llamadas al LLM y otras operaciones en secuencia. Son fundamentales para construir flujos de trabajo complejos.

### Tipos de Cadenas Básicas

- **LLMChain**: La cadena más simple, combina un prompt template con un LLM
- **SimpleSequentialChain**: Ejecuta múltiples cadenas en secuencia, donde la salida de una es la entrada de la siguiente
- **SequentialChain**: Similar pero permite múltiples entradas y salidas

### Caso de Uso: Análisis de Riesgos Laborales

Crearemos una cadena que:
1. Identifica riesgos en una descripción de proceso
2. Genera recomendaciones de seguridad

In [32]:
# Crear la primera cadena simple: LLMChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Definir el template para identificar riesgos
template_riesgos = """
Eres un experto en seguridad industrial. Analiza la siguiente descripción de un proceso
de trabajo y enumera los principales riesgos identificados.

Descripción del proceso:
{descripcion_proceso}

Lista de riesgos identificados (máximo 5):
"""

prompt_riesgos = PromptTemplate(
    input_variables=["descripcion_proceso"],
    template=template_riesgos
)

# Crear la cadena
cadena_identificacion = LLMChain(
    llm=llm,
    prompt=prompt_riesgos,
    output_key="riesgos"  # Nombre de la variable de salida
)

# Probar la cadena
descripcion = """
El operario debe transportar manualmente bidones de 25 kg de solvente químico
desde el almacén hasta la línea de producción, ubicada en el segundo piso.
Debe subir escaleras metálicas sin pasamanos. Una vez en la línea, vierte el
solvente en una mezcladora industrial que opera a 85°C. El área tiene ventilación
natural únicamente.
"""

resultado = cadena_identificacion.invoke({"descripcion_proceso": descripcion})
print("RIESGOS IDENTIFICADOS:")
print(resultado["riesgos"])

/tmp/ipython-input-1570918446.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  cadena_identificacion = LLMChain(


RIESGOS IDENTIFICADOS:
Como experto en seguridad industrial, he analizado la descripción del proceso y he identificado los siguientes riesgos principales:

**Lista de riesgos identificados:**

1.  **Lesiones musculoesqueléticas:** Por la manipulación manual de cargas pesadas (bidones de 25 kg) de forma repetitiva y en condiciones ergonómicamente desfavorables (subir escaleras).
2.  **Caídas a distinto nivel:** Debido al transporte de cargas pesadas por escaleras metálicas que carecen de pasamanos, aumentando significativamente el riesgo de tropiezos, resbalones o pérdida de equilibrio con consecuencias graves.
3.  **Exposición a sustancias químicas:** Por la manipulación y vertido de solventes químicos, con riesgo de inhalación de vapores (agravado por la ventilación natural únicamente y la alta temperatura que aumenta la volatilidad) y contacto dérmico o salpicaduras.
4.  **Incendio, explosión y/o quemaduras:** Por la interacción del solvente químico (potencialmente inflamable) con la

In [33]:
# Crear una segunda cadena para generar recomendaciones
template_recomendaciones = """
Basándote en los siguientes riesgos laborales identificados, genera un plan de
acción con recomendaciones concretas y priorizadas.

Riesgos identificados:
{riesgos}

Plan de acción (enumera 3-5 recomendaciones prioritarias):
"""

prompt_recomendaciones = PromptTemplate(
    input_variables=["riesgos"],
    template=template_recomendaciones
)

cadena_recomendaciones = LLMChain(
    llm=llm,
    prompt=prompt_recomendaciones,
    output_key="plan_accion"
)

# Probar esta cadena con el resultado anterior
resultado2 = cadena_recomendaciones.invoke({"riesgos": resultado["riesgos"]})
print("PLAN DE ACCIÓN:")
print(resultado2["plan_accion"])

PLAN DE ACCIÓN:
Como experto en seguridad industrial, propongo el siguiente plan de acción priorizado para abordar los riesgos identificados, centrándome en la jerarquía de controles (eliminación, sustitución, ingeniería, administrativos, EPP):

---

**Plan de Acción Priorizado**

1.  **Implementación de Controles de Ingeniería para Manipulación de Cargas y Seguridad en Escaleras (Prioridad: Muy Alta)**
    *   **Acciones Concretas:**
        *   **Instalación Inmediata de Pasamanos:** Diseñar e instalar pasamanos robustos y seguros en todas las escaleras metálicas utilizadas para el transporte de cargas, cumpliendo con la normativa aplicable.
        *   **Sistemas de Ayuda Mecánica:** Evaluar e implementar soluciones de ingeniería para la elevación y transporte de bidones de 25 kg, como:
            *   Polipastos eléctricos o manuales con rieles para el ascenso de cargas.
            *   Elevadores de carga o montacargas pequeños si la estructura lo permite.
            *   Carretil

### Análisis: Cadenas Simples

En los ejemplos anteriores ejecutamos dos cadenas manualmente, pasando la salida de una como entrada de la otra. Sin embargo, esto requiere intervención manual. Las cadenas secuenciales automatizan este proceso.

---

## 2.2 Construcción de una Cadena Secuencial Completa

Ahora combinaremos múltiples cadenas en un flujo automático usando SequentialChain.

### Objetivo: Sistema Completo de Análisis de Seguridad

El sistema realizará:
1. Identificación de riesgos
2. Generación de recomendaciones
3. Estimación de costos de implementación
4. Generación de resumen ejecutivo

In [34]:
# Construir un sistema de análisis de seguridad completo con SequentialChain
from langchain.chains import SequentialChain

# Cadena 1: Identificación de riesgos (ya la tenemos)
# cadena_identificacion

# Cadena 2: Generación de recomendaciones (ya la tenemos)
# cadena_recomendaciones

# Cadena 3: Estimación de costos
template_costos = """
Basándote en el siguiente plan de acción de seguridad, estima el costo aproximado
de implementación en dólares estadounidenses. Considera: equipos, capacitación,
modificaciones de infraestructura, etc.

Plan de acción:
{plan_accion}

Estimación de costos (desglosada por rubro):
"""

prompt_costos = PromptTemplate(
    input_variables=["plan_accion"],
    template=template_costos
)

cadena_costos = LLMChain(
    llm=llm,
    prompt=prompt_costos,
    output_key="estimacion_costos"
)

# Cadena 4: Resumen ejecutivo
template_resumen_exec = """
Genera un resumen ejecutivo breve (máximo 100 palabras) que integre:

Riesgos:
{riesgos}

Plan de acción:
{plan_accion}

Costos estimados:
{estimacion_costos}

Resumen ejecutivo:
"""

prompt_resumen_exec = PromptTemplate(
    input_variables=["riesgos", "plan_accion", "estimacion_costos"],
    template=template_resumen_exec
)

cadena_resumen = LLMChain(
    llm=llm,
    prompt=prompt_resumen_exec,
    output_key="resumen_ejecutivo"
)

# Crear la cadena secuencial que integra todas las cadenas
cadena_completa = SequentialChain(
    chains=[
        cadena_identificacion,
        cadena_recomendaciones,
        cadena_costos,
        cadena_resumen
    ],
    input_variables=["descripcion_proceso"],
    output_variables=["riesgos", "plan_accion", "estimacion_costos", "resumen_ejecutivo"],
    verbose=True  # Mostrar el progreso de cada paso
)

print("Cadena secuencial completa construida exitosamente")

Cadena secuencial completa construida exitosamente


In [35]:
# Ejecutar la cadena completa con un nuevo caso
descripcion_caso2 = """
El proceso de pintado automotriz requiere que el operario ingrese a una cabina
cerrada donde se aplica pintura mediante pistola neumática. La temperatura dentro
de la cabina alcanza 40°C. El operario trabaja turnos de 8 horas continuas.
Los vapores de la pintura contienen solventes orgánicos volátiles. La cabina
tiene extracción de aire que funciona intermitentemente.
"""

# Ejecutar todas las cadenas en secuencia
resultado_completo = cadena_completa.invoke({"descripcion_proceso": descripcion_caso2})

# Mostrar resultados organizados
print("\n" + "="*70)
print("INFORME COMPLETO DE ANÁLISIS DE SEGURIDAD")
print("="*70)

print("\n1. RIESGOS IDENTIFICADOS:")
print("-" * 70)
print(resultado_completo["riesgos"])

print("\n2. PLAN DE ACCIÓN:")
print("-" * 70)
print(resultado_completo["plan_accion"])

print("\n3. ESTIMACIÓN DE COSTOS:")
print("-" * 70)
print(resultado_completo["estimacion_costos"])

print("\n4. RESUMEN EJECUTIVO:")
print("-" * 70)
print(resultado_completo["resumen_ejecutivo"])



> Entering new SequentialChain chain...

> Finished chain.

INFORME COMPLETO DE ANÁLISIS DE SEGURIDAD

1. RIESGOS IDENTIFICADOS:
----------------------------------------------------------------------
Como experto en seguridad industrial, he analizado la descripción del proceso de pintado automotriz y he identificado los siguientes riesgos principales:

**Lista de riesgos identificados:**

1.  **Exposición a sustancias químicas peligrosas (SOV):** La inhalación y el contacto dérmico con los solventes orgánicos volátiles presentes en los vapores de la pintura son un riesgo crítico, exacerbado por la cabina cerrada y la ventilación intermitente.
2.  **Estrés térmico por altas temperaturas:** La exposición continua a 40°C durante 8 horas puede causar deshidratación, golpes de calor, agotamiento y disminución de la capacidad de concentración del operario.
3.  **Ventilación deficiente y acumulación de contaminantes:** El funcionamiento intermitente del sistema de extracción permite la conc

## Caso Integrador: Sistema RAG Completo

Ahora integraremos todo lo aprendido en un sistema RAG (Retrieval Augmented Generation) completo que:

1. Carga documentos sobre normativas
2. Los divide y almacena en una base vectorial
3. Responde preguntas buscando información relevante
4. Retorna respuestas estructuradas con citas de fuentes

In [36]:
# Crear una base de conocimiento sobre normativas de construcción
documentos_normativas = [
    """
    Norma NTC-1500: Código Colombiano de Construcciones Sismo Resistentes

    Artículo 12.3 - Cargas de viento
    Las estructuras deben diseñarse para resistir cargas de viento calculadas según
    la velocidad básica de diseño establecida para la región. En zonas costeras, la
    velocidad básica mínima es de 120 km/h. Los edificios de más de 40 metros de
    altura requieren estudios de túnel de viento.
    """,
    """
    Norma NTC-2505: Instalaciones Eléctricas en Edificaciones

    Sección 7.2 - Protección contra sobrecorrientes
    Todos los circuitos derivados deben protegerse mediante interruptores automáticos
    o fusibles calibrados según la capacidad del conductor. La corriente nominal del
    dispositivo de protección no debe exceder el 80% de la capacidad del conductor
    para cargas continuas. Los tableros de distribución deben ubicarse en áreas
    accesibles y ventiladas.
    """,
    """
    Norma NTC-3631: Ventilación y Calidad de Aire Interior

    Artículo 4.5 - Renovación de aire
    Los espacios ocupados requieren una tasa mínima de renovación de aire de
    15 litros por segundo por persona. En áreas de trabajo con generación de
    contaminantes, la tasa debe incrementarse según el análisis de riesgo químico.
    Los sistemas de ventilación mecánica deben someterse a mantenimiento trimestral
    con limpieza de ductos y reemplazo de filtros.
    """,
    """
    Norma NTC-4595: Accesibilidad al Medio Físico

    Capítulo 3 - Rampas de acceso
    Las rampas para personas con movilidad reducida deben tener pendiente máxima de
    8.33% (1:12) para longitudes hasta 9 metros. Pendientes mayores requieren
    descansos intermedios cada 9 metros. El ancho mínimo libre es de 1.20 metros.
    Las rampas deben contar con pasamanos a ambos lados a alturas de 90 cm y 70 cm.
    """,
    """
    Norma NTC-5613: Gestión de Residuos de Construcción y Demolición

    Artículo 8 - Clasificación y almacenamiento
    Los residuos deben clasificarse en obra en categorías: inertes (concreto, ladrillos),
    reciclables (metales, maderas), peligrosos (asbesto, pinturas). El almacenamiento
    temporal no debe exceder 30 días. Los contenedores deben estar etiquetados y
    ubicados en zonas que no obstruyan circulación.
    """
]

# Procesar los documentos
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Dividir documentos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

docs_normativas_split = text_splitter.create_documents(documentos_normativas)

# Crear base de datos vectorial
vectorstore_normativas = Chroma.from_documents(
    documents=docs_normativas_split,
    embedding=embeddings_model,
    collection_name="normativas_construccion"
)

print(f"Base de conocimiento creada con {len(docs_normativas_split)} fragmentos")

Base de conocimiento creada con 10 fragmentos


In [37]:
# Crear una cadena RAG completa usando RetrievalQA
from langchain.chains import RetrievalQA

# Configurar el retriever
retriever = vectorstore_normativas.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Recuperar los 3 fragmentos más relevantes
)

# Crear la cadena RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" coloca todos los documentos en un solo prompt
    retriever=retriever,
    return_source_documents=True,  # Retornar las fuentes consultadas
    verbose=True
)

print("Sistema RAG configurado exitosamente")

Sistema RAG configurado exitosamente


In [38]:
# Realizar consultas al sistema RAG
pregunta1 = "¿Cuáles son los requisitos para las rampas de accesibilidad?"

resultado_rag = qa_chain.invoke({"query": pregunta1})

print("\n" + "="*70)
print("CONSULTA:", pregunta1)
print("="*70)
print("\nRESPUESTA:")
print(resultado_rag["result"])
print("\n" + "-"*70)
print("DOCUMENTOS FUENTE CONSULTADOS:")
print("-"*70)
for i, doc in enumerate(resultado_rag["source_documents"], 1):
    print(f"\nFuente {i}:")
    print(doc.page_content[:200] + "...")



> Entering new RetrievalQA chain...

> Finished chain.

CONSULTA: ¿Cuáles son los requisitos para las rampas de accesibilidad?

RESPUESTA:
Los requisitos para las rampas de accesibilidad, según la información proporcionada, son:

*   **Pendiente:**
    *   Máxima de 8.33% (1:12) para longitudes de hasta 9 metros.
    *   Si la pendiente es mayor, se requieren descansos intermedios cada 9 metros.
*   **Ancho:**
    *   El ancho mínimo libre es de 1.20 metros.
*   **Pasamanos:**
    *   Deben contar con pasamanos a ambos lados.
    *   A dos alturas: 90 cm y 70 cm.
    *   Deben estar ubicados en zonas que no obstruyan la circulación.

----------------------------------------------------------------------
DOCUMENTOS FUENTE CONSULTADOS:
----------------------------------------------------------------------

Fuente 1:
Las rampas deben contar con pasamanos a ambos lados a alturas de 90 cm y 70 cm....

Fuente 2:
Norma NTC-4595: Accesibilidad al Medio Físico
    
    Capítulo 3 - Rampas de 

In [39]:
# Realizar más consultas
preguntas_adicionales = [
    "¿Qué dice la normativa sobre protección eléctrica?",
    "¿Cuál es la tasa mínima de renovación de aire requerida?",
    "¿Cómo deben gestionarse los residuos de construcción?"
]

for pregunta in preguntas_adicionales:
    resultado = qa_chain.invoke({"query": pregunta})
    print("\n" + "="*70)
    print(f"PREGUNTA: {pregunta}")
    print("="*70)
    print(resultado["result"])
    print()



> Entering new RetrievalQA chain...

> Finished chain.

PREGUNTA: ¿Qué dice la normativa sobre protección eléctrica?
La Norma NTC-2505, en su Sección 7.2 - Protección contra sobrecorrientes, establece que:

*   Todos los circuitos derivados deben protegerse mediante interruptores automáticos o fusibles.
*   Estos dispositivos deben estar calibrados según la capacidad del conductor.
*   La corriente nominal del dispositivo de protección no debe exceder el 80% de la capacidad del conductor para cargas continuas.



> Entering new RetrievalQA chain...

> Finished chain.

PREGUNTA: ¿Cuál es la tasa mínima de renovación de aire requerida?
La tasa mínima de renovación de aire requerida es de 15 litros por segundo por persona.



> Entering new RetrievalQA chain...

> Finished chain.

PREGUNTA: ¿Cómo deben gestionarse los residuos de construcción?
Según la Norma NTC-5613, los residuos de construcción deben gestionarse de la siguiente manera:

*   **Clasificación:** Deben clasificarse en obr

---

## Conclusiones y Buenas Prácticas

### Resumen de Conceptos Clave

1. **Document Loaders**: Permiten cargar datos desde múltiples fuentes (PDF, CSV, web, APIs)
2. **Text Splitters**: Dividen documentos largos manteniendo coherencia semántica
3. **Embeddings**: Representaciones vectoriales que capturan significado semántico
4. **Vector Stores**: Bases de datos optimizadas para búsqueda por similitud
5. **Compresión Contextual**: Optimiza la recuperación extrayendo solo información relevante
6. **Cadenas**: Automatizan flujos de trabajo multi-paso
7. **Output Parsers**: Estructuran las respuestas del LLM para integración con sistemas

### Buenas Prácticas

**Gestión de Documentos:**
- Utiliza chunk_overlap para evitar perder contexto en los límites
- Ajusta chunk_size según la complejidad del contenido (textos técnicos = fragmentos más grandes)
- Mantén metadatos ricos (fuente, fecha, autor) para trazabilidad

**Búsqueda y Recuperación:**
- Experimenta con diferentes valores de k (número de documentos recuperados)
- Considera usar hybrid search (combinación de búsqueda semántica y por palabras clave)
- Implementa compresión contextual cuando trabajes con documentos muy largos

**Prompts y Cadenas:**
- Serializa prompts importantes para control de versiones
- Usa output parsers cuando necesites integración con otros sistemas
- Diseña cadenas modulares para facilitar mantenimiento

**Optimización:**
- Monitorea el uso de tokens para controlar costos
- Implementa caché cuando sea apropiado
- Considera modelos más pequeños para tareas simples

---

## Ejercicios Propuestos

### Ejercicio 1: Análisis de Documentos Personalizados
Carga un documento PDF de tu elección (artículo académico, manual técnico, etc.) y:
1. Divídelo en fragmentos apropiados
2. Crea una base vectorial
3. Implementa un sistema de preguntas y respuestas

### Ejercicio 2: Cadena Multi-Idioma
Diseña una cadena secuencial que:
1. Reciba un texto en español
2. Lo traduzca al inglés
3. Genere un resumen en inglés
4. Traduzca el resumen de vuelta al español

### Ejercicio 3: Sistema de Recomendaciones
Construye un sistema que:
1. Cargue descripciones de productos/servicios
2. Reciba una consulta del usuario
3. Recupere los 5 productos más relevantes
4. Genere una recomendación personalizada explicando por qué son relevantes

### Ejercicio 4: Parser Personalizado
Crea un output parser que retorne información estructurada específica de tu dominio (por ejemplo: datos médicos, especificaciones técnicas, análisis financiero).

---

## Recursos Adicionales

### Documentación Oficial
- LangChain Documentation: https://python.langchain.com/
- Google Generative AI: https://ai.google.dev/

### Conceptos para Profundizar
- RAG avanzado: Re-ranking, Hybrid search
- Agentes: Sistemas que pueden usar herramientas y tomar decisiones
- Memory: Mantener contexto entre conversaciones
- Evaluation: Métricas para evaluar sistemas RAG

### Próximos Pasos
1. Explorar agentes con herramientas personalizadas
2. Implementar sistemas multi-agente
3. Integrar con bases de datos SQL
4. Construir interfaces de usuario con Streamlit o Gradio

---

## Cierre del Taller

Has completado un recorrido completo por los fundamentos de LangChain, desde la carga de documentos hasta la construcción de sistemas RAG complejos. Los conceptos aprendidos son la base para construir aplicaciones de IA generativa de nivel profesional.

Recuerda que la práctica constante es clave. Experimenta con diferentes configuraciones, prueba con tus propios datos y adapta estos patrones a tus casos de uso específicos.

¡Éxito en tu camino como desarrollador de sistemas basados en LLMs!